# Reproduction Notebook

This notebook is title as is, due to the reproduction needed for this course.

In [9]:
import pandas as pd
import numpy as np

import sys
import os
import pathlib
import glob

import netCDF4

sys.path.append("src/")

from visualization import mpsettings
import matplotlib.pyplot as plt
import shap

from catboost import CatBoostRegressor

## Global varriables

In [2]:
PATH = "data/"
fpath ="data"
experiment = "2005-2012"

# Why is it 2005-2012?

In [29]:
path0 = "data/test/"
data1993 = netCDF4.Dataset(path0 + "dt_global_twosat_phy_l4_19930101_vDT2018.nc")
data2011 = netCDF4.Dataset(path0 + "dt_global_twosat_phy_l4_20110101_vDT2018.nc")
data2018 = netCDF4.Dataset(path0 + "dt_global_twosat_phy_l4_20180101_vDT2018.nc")

dim0 = data1993.dimensions.values()
dim1 = data2011.dimensions.values()
dim2 = data2018.dimensions.values()





"""
print("\nVariables in dataset from 2018")
for j in vars2:
    print(j)
"""

print("\nDimensions in dataset from 1993")

for k in dim0:
    print(k)

print("\nDimensions in dataset from 2011")

for i in dim1:
    print(i)

print("\nDimensions in dataset from 2018")
for j in dim2:
    print(j)
    
print("\nVariables in dataset from 2011")


    
vars1 = data2011.variables.values()
vars2 = data2018.variables.values()

for i in vars2:
    print("\n")
    print(i)


Dimensions in dataset from 1993
<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 1
<class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 720
<class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 1440
<class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 2

Dimensions in dataset from 2011
<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 1
<class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 720
<class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 1440
<class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 2

Dimensions in dataset from 2018
<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 1
<class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 720
<class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 1440
<class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 2

Variables in dataset from 2011


<class 'netCDF4._netCDF4.Variable'>
int32 crs()
    comment: This is a container variab

I remember to Bence saying they changed formats. I don't really know whats going on, but these seem to be pretty consistent shapes, even the variables to align with each other in ranges and types. Now I am really curious if I could extent his work on the available data

### Let's continue

In [25]:
Eddy = pd.read_csv(
    PATH +
    fpath+".csv"
    )

In [26]:
display(Eddy.info())
display(Eddy.head())
display(Eddy.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2258 entries, 0 to 2257
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   2258 non-null   int64  
 1   longitude    2258 non-null   float64
 2   latitude     2258 non-null   float64
 3   adt          2258 non-null   float64
 4   ugos         2258 non-null   float64
 5   vgos         2258 non-null   float64
 6   sla          2258 non-null   float64
 7   ugosa        2258 non-null   float64
 8   vgosa        2258 non-null   float64
 9   Born         2258 non-null   object 
 10  Dead         2258 non-null   object 
 11  Errcount     2258 non-null   int64  
 12  Longevity    2258 non-null   int64  
 13  radius       2258 non-null   float64
 14  radius_glob  2258 non-null   float64
dtypes: float64(10), int64(3), object(2)
memory usage: 264.7+ KB


None

,Unnamed: 0,longitude,latitude,adt,ugos,vgos,sla,ugosa,vgosa,Born,Dead,Errcount,Longevity,radius,radius_glob
0,1179,309.625,0.375,0.6791,0.3801,-0.9722,-0.0210,0.3387,-1.0618,2005-01-05,2005-01-06 00:00:00,0,1,1.3523,1.4005
1,1433,310.875,1.875,0.4370,-1.3249,-0.4032,-0.1056,-2.2400,-0.2611,2005-01-01,2005-01-07 00:00:00,0,6,1.7281,2.5011
2,1616,310.625,2.375,0.5589,-0.8862,-3.3199,0.0537,-1.5675,-3.2786,2005-01-01,2005-01-08 00:00:00,0,7,4.2061,4.8461
3,1617,310.625,2.125,0.5164,-2.2212,-3.4013,-0.0072,-3.0858,-3.2967,2005-01-01,2005-01-08 00:00:00,0,7,5.6225,6.3825
4,1636,310.875,2.125,0.4789,-1.3860,-1.0049,-0.0421,-2.1623,-0.9566,2005-01-01,2005-01-08 00:00:00,0,7,2.3909,3.1189


,Unnamed: 0,longitude,latitude,adt,ugos,vgos,sla,ugosa,vgosa,Born,Dead,Errcount,Longevity,radius,radius_glob
2253,1220611,309.625,0.125,0.7132,0.0831,0.7162,0.0138,0.0755,0.6182,2011-12-26,2011-12-28 00:00:00,0,2,0.7993,0.6937
2254,1220683,124.375,-16.375,1.1778,-0.3083,0.3046,0.1703,-0.2876,0.2581,2011-05-07,2011-12-28 00:00:00,0,235,0.6129,0.5457
2255,1220779,309.125,-0.125,0.7264,0.0722,0.8272,0.0243,0.0697,0.6368,2011-12-26,2011-12-28 00:00:00,0,2,0.8994,0.7065
2256,1220886,309.375,-0.125,0.7247,0.0682,0.8194,0.0225,0.0660,0.6617,2011-12-26,2011-12-28 00:00:00,0,2,0.8876,0.7277
2257,1220887,309.375,0.125,0.7228,0.0767,0.7904,0.0217,0.0743,0.6637,2011-12-26,2011-12-28 00:00:00,0,2,0.8671,0.7380
